### More information

Getting more information based on the book links I got from previous script
- use the book link to get catergory, publisher, pages, isbn, datepublished,e,g
- get the data, save them to pickle file

In [1]:
from bs4 import BeautifulSoup  ###import necessary libraries
from numpy import random
from time import sleep
from datetime import datetime
import requests
import pandas as pd
import os
import csv


def write_to_pickle(name, data):  ### his function takes whatever list/dict into dataframe and convert it into a pickle file
    file_name = f'{name}.pkl'
    
    if file_name in os.listdir():
        return 'File already exist'

    df = pd.DataFrame(data)
    df.to_pickle(file_name)

## Get Extra Information
After getting all the url for each book,use the url to get extra Information, e.g reviews, publisher

In [ ]:
print(datetime.now())


def find_text(bs_find):  ### This function find text in the 
    return bs_find.text if bs_find else None


def more_info(each_book_url):  ### This function takes book link as input
    r = requests.get(each_book_url)
    link_page = BeautifulSoup(r.text)
    book = {}

    book['categories'] = find_text(link_page.find('div', class_='breadcrumbs span12'))
    book['contributors'] = find_text(link_page.find('span',class_='contributors'))
    book['pages'] = find_text(link_page.find('span', itemprop='numberOfPages'))
    book['datePublished'] = link_page.find(itemprop='datePublished').get('content')
    book['availability'] = find_text(link_page.find('span', id='scope_offer_availability'))
    book['description'] = find_text(link_page.find('div', itemprop='description'))
    book['publisher'] = find_text(link_page.find('span', itemprop='publisher'))
    book['isbn'] = find_text(link_page.find('span',itemprop='isbn'))
    book['weight'] = find_text(link_page.find('span',itemprop='weight'))
    book['num_reviews'] = link_page.find(itemprop='reviewCount').get('content')
    book['num_stars'] = link_page.find(itemprop='ratingValue').get('content')
    book['media_reviews'] = find_text(link_page.find('div', class_='show-desc'))


    if book['num_reviews']:
        a_href= link_page.find('div', id='reviews').find("a", href=True)
        book['review_links'] = 'https://www.waterstones.com' + a_href['href']
        
        
    book['book_link'] = each_book_url

    return book

In [ ]:
x = r'../Data/books_basic.pkl'  ### take in the data got from perivous
df = pd.read_pickle(x) 
urls = df['book_link'].to_list()  ### book_link to a list

book_details = []
for idx, url in enumerate(urls):
    try:  
        book_details.append(more_info(url))
        if (idx + 1) % 10000 == 0:  ### every 10000 books write to pickle
            write_to_pickle(f'books_{idx+1}', book_details) 
            book_details = []

    except Exception as err:  ### catch the errors
        with open('errors.txt', 'a') as f:
            f.write(f'{idx}:{err}\n')

print('FINISH:', datetime.now())